---
Atanda Abdullahi Adewale
This code is not fully functional as stabilized video is not generated yet  
---

The Program stabilizes a video by applying affine transformation to each frame based on the motion information between the current frame and the previous frame, and then warping the current frame using the estimated affine transformation.


- affine_motionfunction returns the estimated affine transformation matrix.

- compute_flow_fields() returns flow fields represent the pixel displacements in the horizontal and vertical directions.

- image_warp() function returns the warped image.

- The function write_video() writes the list of warped images to a video of a give FPS.

- The function stabilize_frames() computes the flow fields for each pixel between the previous and current frames using the affine   motion estimation and motion compensation. The accumulated flow fields are updated using the new flow fields. The function returns the warped current frame, the updated previous grayscale image, and the updated accumulated flow fields.

In [1]:
import cv2
import os
import numpy as np
from scipy import ndimage
import matplotlib.pyplot as plt

In [2]:
def affine_motion(I1, I2, height, width):
    A = np.zeros((height * width * 2, 6))
    b = np.zeros((height * width * 2, 1))

    Gx = np.array([[-1, 1], [-1, 1]])
    Gy = np.array([[-1, -1], [1, 1]])
    Gtk = np.array([[-1, -1], [-1, -1]])
    Gtk1 = np.array([[1, 1], [1, 1]])

    Ix = ndimage.convolve(I1, Gx) + ndimage.convolve(I2, Gx)
    Iy = ndimage.convolve(I1, Gy) + ndimage.convolve(I2, Gy)
    It = ndimage.convolve(I1, Gtk) + ndimage.convolve(I2, Gtk1)

    count = 0
    for i in range(height):
        for j in range(width):
            A[count] = [j, i, 1, 0, 0, 0]
            A[height * width + count] = [0, 0, 0, j, i, 1]
            b[count] = -It[i, j]
            count += 1

    ATA = np.dot(A.T, A)
    ATA_inv = np.linalg.inv(ATA)
    ATb = np.dot(A.T, b)
    affine_params = np.dot(ATA_inv, ATb)

    return affine_params

In [3]:
def compute_flow_fields(theta, width, height):
    x, y = np.meshgrid(np.arange(width), np.arange(height))
    x = x.flatten()
    y = y.flatten()

    M = np.array([[theta[0], theta[1], theta[2]], [theta[3], theta[4], theta[5]]])
    v = np.dot(np.vstack((x, y, np.ones_like(x))).T, M.T)
    flow_x = v.T[0].reshape((height, width))
    flow_y = v.T[1].reshape((height, width))

    return flow_x, flow_y

In [4]:
def image_warp(I, U, V):
    # create pixel coordinates
    x = np.linspace(0, I.shape[1]-1, I.shape[1])
    y = np.linspace(0, I.shape[0]-1, I.shape[0])
    xi, yi = np.meshgrid(x, y)
    
    # warping image intensities to the new locations
    warped_img = cv2.remap(I, (xi+U).astype(np.float32), (yi+V).astype(np.float32), interpolation=cv2.INTER_LINEAR)
    return warped_img

In [5]:
def write_video(warped_list, video_name, FPS=15):
    img = warped_list[0]
    # create a video with the provided FPS
    video_writer_warp = cv2.VideoWriter('stabilized_video_' + video_name + '.mp4', cv2.VideoWriter_fourcc(*'MP4V'), FPS, (img.shape[1], img.shape[0]))
    for i in range(len(warped_list)):
        video_writer_warp.write(cv2.cvtColor(warped_list[i], cv2.COLOR_GRAY2BGR))
    video_writer_warp.release()

In [6]:
# function to stabilize a pair of frames
def stabilize_frames(prev_frame, curr_frame, prev_gray, height, width, U0, V0):
    # Convert current frame to grayscale
    curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
    
    # Compute flow fields for all pixels using previous flow fields and affine motion parameters
    flow_x, flow_y = compute_flow_fields(theta, curr_gray.shape[1], curr_gray.shape[0], U0, V0)
    
    # Warp previous frame using flow fields
    prev_warped = image_warp(prev_gray, flow_x, flow_y)
    
    # Compute affine motion parameters between warped previous frame and current frame
    theta = affine_motion(prev_warped, curr_gray, height, width)
    
    # Compute flow fields for all pixels using affine motion parameters
    flow_x, flow_y = compute_flow_fields(theta, curr_gray.shape[1], curr_gray.shape[0])
    
    # Warp current frame using flow fields
    curr_warped = image_warp(curr_frame, flow_x, flow_y)
    
    # Update accumulated flow fields
    U0 += flow_x
    V0 += flow_y
    
    # Set current frame as previous frame for next iteration
    prev_gray = curr_gray.copy()
    
    return curr_warped, prev_gray, U0, V0


In [7]:
# Define paths to the input images and output folder
input_path = 'taxi/'
output_path = 'output/'

# Create output folder if it doesn't exist
if not os.path.exists(output_path):
    os.makedirs(output_path)

# Load the first image and convert to grayscale
prev_frame = cv2.imread(os.path.join(input_path, '11.jpg'))
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# Get the height and width of the images
height, width = prev_gray.shape

# Initialize accumulated flow fields
U0 = np.zeros((height, width))
V0 = np.zeros((height, width))

# Initialize list of stabilized frames
stabilized_frames = [prev_gray]

In [8]:
# Loop over all subsequent images in the sequence
for i in range(12, 42):
    # Load current image and convert to grayscale
    curr_frame = cv2.imread(os.path.join(input_path, f'{i}.jpg'))
    curr_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
    
    # Compute affine motion parameters between current and previous frame
    theta = affine_motion(prev_gray, curr_gray, height, width)

    
    # Compute flow fields for all pixels using affine motion parameters
    flow_x, flow_y = compute_flow_fields(theta, curr_gray.shape[1], curr_gray.shape[0])
    
    # Accumulate flow fields
    U0 += flow_x
    V0 += flow_y

    # Stabilize current frame
    curr_stabilized = image_warp(curr_gray, U0, V0)

    # Add stabilized frame to list
    stabilized_frames.append(curr_stabilized)

    # Set current frame as previous frame for next iteration
    prev_gray = curr_gray.copy()

    # Generate and save stabilized video
    write_video(stabilized_frames, 'taxi')
    
print('Stabilization completed!')

Stabilization completed!
